# Simple Starter Notebook for;
## UmojaHack Africa 2021 #2: Sendy - Delivery Rider Response Challenge by UmojaHack Africa

Can you predict who is the best delivery rider for an order placed via logistics company Sendy?

![Umoja Hack](https://zindpublic.blob.core.windows.net/public/uploads/competition/image/152/thumb_c5ec4e2a-e000-4176-a93c-dd1143c2b60f.png)

The objective of this challenge is to create a machine learning model that will predict whether a rider will accept, decline or ignore an order sent to them.

# Importation et visualisation des données

In [2]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

In [3]:
# Load files
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
riders = pd.read_csv('Riders.csv')
ss = pd.read_csv('SampleSubmission.csv')

In [4]:
# check data shapes
train.shape, test.shape, riders.shape, ss.shape

((179867, 21), (76791, 20), (2632, 4), (76791, 2))

In [5]:
# Preview train
train.head()

,ID,order_id,dispatch_day,dispatch_day_of_week,dispatch_time,client_id,client_type,order_license_status,order_carrier_type,vendor_type,...,rider_license_status,rider_carrier_type,rider_amount,rider_lat,rider_long,pickup_lat,pickup_long,drop_off_lat,drop_off_long,target
0,ID_SCUW21PVAU,4435,27,6,09:02:54,593630,Business,0,2,Bike,...,0,1,1080,-42.698343,-17.228539,-42.692371,-17.248305,-42.687442,-17.424682,1
1,ID_2HA7X30JMN,32711,30,7,13:01:37,837729,Personal,0,1,Bike,...,0,1,730,-42.787317,-17.288252,-42.784046,-17.290121,-42.673267,-17.234595,2
2,ID_IAJWDTBY6M,8712,14,2,10:01:00,695129,Personal,0,2,Bike,...,1,1,490,-42.749180,-17.287848,-42.765204,-17.293784,-42.813953,-17.294805,1
3,ID_LKSVPNYMTR,44869,22,3,14:11:16,1504660,Personal,0,2,Bike,...,1,1,510,-42.836266,-17.311920,-42.831913,-17.315311,-42.812409,-17.265441,2
4,ID_O7N8Y918YH,57590,27,5,16:11:38,36869,Business,0,2,Bike,...,0,0,400,-42.828195,-17.322818,-42.836056,-17.318111,-42.828517,-17.302052,0


In [6]:
# Preview riders
riders.head()

,Rider ID,Active Rider Age,Average Partner Rating,Number of Ratings
0,16261,308,21.05,321
1,8832,224,10.00,27
2,53866,238,17.76,25
3,46368,343,24.56,320
4,45609,399,14.97,214


In [7]:
# Merge rider dataset to train and test sets
train = train.merge(riders, how = 'left', left_on='rider_id', right_on='Rider ID')
test = test.merge(riders, how = 'left', left_on='rider_id', right_on='Rider ID')

# Preview merged dataframe
train.head()

,ID,order_id,dispatch_day,dispatch_day_of_week,dispatch_time,client_id,client_type,order_license_status,order_carrier_type,vendor_type,...,rider_long,pickup_lat,pickup_long,drop_off_lat,drop_off_long,target,Rider ID,Active Rider Age,Average Partner Rating,Number of Ratings
0,ID_SCUW21PVAU,4435,27,6,09:02:54,593630,Business,0,2,Bike,...,-17.228539,-42.692371,-17.248305,-42.687442,-17.424682,1,30153,11,10.00,1
1,ID_2HA7X30JMN,32711,30,7,13:01:37,837729,Personal,0,1,Bike,...,-17.288252,-42.784046,-17.290121,-42.673267,-17.234595,2,20884,68,24.13,229
2,ID_IAJWDTBY6M,8712,14,2,10:01:00,695129,Personal,0,2,Bike,...,-17.287848,-42.765204,-17.293784,-42.813953,-17.294805,1,33143,273,24.92,123
3,ID_LKSVPNYMTR,44869,22,3,14:11:16,1504660,Personal,0,2,Bike,...,-17.311920,-42.831913,-17.315311,-42.812409,-17.265441,2,96531,168,23.76,175
4,ID_O7N8Y918YH,57590,27,5,16:11:38,36869,Business,0,2,Bike,...,-17.322818,-42.836056,-17.318111,-42.828517,-17.302052,0,103546,95,24.53,42


In [11]:
train.describe()

,order_id,dispatch_day,dispatch_day_of_week,client_id,order_license_status,order_carrier_type,rider_id,rider_license_status,rider_carrier_type,rider_amount,...,rider_long,pickup_lat,pickup_long,drop_off_lat,drop_off_long,target,Rider ID,Active Rider Age,Average Partner Rating,Number of Ratings
count,179867.000000,179867.000000,179867.000000,1.798670e+05,179867.000000,179867.000000,179867.000000,179867.000000,179867.000000,179867.000000,...,179867.000000,179867.000000,179867.000000,179867.000000,179867.000000,179867.000000,179867.000000,179867.000000,179867.000000,179867.000000
mean,32639.833149,16.604903,4.185198,4.654108e+05,0.239555,1.619007,70389.969944,0.589936,0.868731,517.544185,...,-17.286813,-42.817156,-17.286229,-42.814879,-17.282547,0.652843,70389.969944,141.981497,17.250388,82.793948
std,18227.428596,8.528368,1.683574,6.634234e+05,0.426813,0.536552,33981.806096,0.491846,0.337695,157.385430,...,0.100707,0.106425,0.100101,0.114158,0.103640,0.791099,33981.806096,90.447924,6.250999,103.392784
min,100.000000,1.000000,1.000000,2.610000e+02,0.000000,0.000000,1817.000000,0.000000,0.000000,200.000000,...,-20.079067,-45.725110,-20.070241,-45.793523,-20.352897,0.000000,1817.000000,0.000000,1.000000,0.000000
25%,16875.000000,9.000000,3.000000,7.564700e+04,0.000000,1.000000,40756.000000,0.000000,1.000000,410.000000,...,-17.301329,-42.829119,-17.299970,-42.830628,-17.301066,0.000000,40756.000000,70.000000,10.000000,12.000000
50%,32964.000000,17.000000,4.000000,1.434280e+05,0.000000,2.000000,73485.000000,1.000000,1.000000,460.000000,...,-17.286776,-42.820037,-17.283554,-42.806976,-17.282880,0.000000,73485.000000,134.000000,19.020000,49.000000
75%,48117.000000,24.000000,6.000000,5.936300e+05,0.000000,2.000000,100441.000000,1.000000,1.000000,570.000000,...,-17.264774,-42.789742,-17.263246,-42.783811,-17.259310,1.000000,100441.000000,196.000000,22.970000,110.000000
max,64427.000000,31.000000,7.000000,2.568456e+06,1.000000,2.000000,123694.000000,1.000000,1.000000,2190.000000,...,-15.480446,-40.699055,-15.482671,-40.683763,-15.324765,2.000000,123694.000000,399.000000,24.960000,637.000000


In [19]:
len(set(train.order_id))

58848

In [15]:
'dispatch_day', 'dispatch_day_of_week',
       'dispatch_time', 'client_id', 'client_type', 'order_license_status',
       'order_carrier_type', 'vendor_type', 'rider_id', 'rider_license_status',
       'rider_carrier_type', 'rider_amount', 'rider_lat', 'rider_long',
       'pickup_lat', 'pickup_long', 'drop_off_lat', 'drop_off_long', 'target',
       'Rider ID', 'Active Rider Age', 'Average Partner Rating',
       'Number of Ratings'

5141

# Traitement des données

In [7]:
# One hot encoding
train = pd.get_dummies(train, columns=['client_type', 'vendor_type'])
test = pd.get_dummies(test, columns=['client_type', 'vendor_type'])

In [8]:
# Split data
main_cols = train.columns.difference(['ID', 'order_id', 'rider_id', 'Rider ID', 'target', 'dispatch_time',	'client_id']).tolist()
X = train[main_cols]
y = train.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 3031)

# Train a model
model = LGBMClassifier(random_state=3031)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Check score
accuracy_score(y_test, y_pred)

0.6144437649413466

In [9]:
# Make predictions in test set and prepare submission file
predictions = model.predict(test[main_cols])
sub_file = ss.copy()
sub_file.target = predictions
sub_file.to_csv('Baseline.csv', index = False)